![Image_jeu_Mastermind](https://upload.wikimedia.org/wikipedia/commons/2/2d/Mastermind.jpg)

# Règle du jeu

Le jeu se joue à deux : un codificateur et un décodeur.
Le but est de deviner, par déductions successives, la couleur de chacun des 4 pions cachés derrière un écran.
Les couleurs possibles sont au nombre de 6 (rouge, vert, bleu, jaune, orange, violet).

Déroulement du jeu : le codificateur pose à l'endroit prévu à cet effet, les 4 pions de son choix. Il doit prendre soin de ne pas révéler la couleur et la répartition dans les trous des pions. Rien ne l'empêche d'en choisir plusieurs d'une même couleur.

Son adversaire, le décodeur, est chargé de déchiffrer ce code secret. Il doit le faire en 10 coups maximum. Il place 4 ou 5 pions dans les trous de la première rangée immédiatement près de lui. Si l'un des pions correspond par sa position et sa couleur à un pion caché derrière l'écran, le codificateur l'indique en plaçant une fiche noire dans l'un des trous de marque, sur le côté droit correspondant du plateau. Si l'un des pions correspond uniquement par sa couleur, le Codificateur l'indique par une fiche blanche dans l'un des trous de marque. S'il n'y a aucune correspondance, il ne marque rien.

Le décodeur continue de poser des pions par rangées successives. La tactique consiste à sélectionner en fonction des coups précédents, couleurs et positions, de manière à obtenir le maximum d'informations de la réponse du partenaire puisque le nombre de propositions est limité par le nombre de rangées de trous du jeu. Dans la plupart des cas, il s'efforce de se rapprocher le plus possible de la solution, compte tenu des réponses précédentes, mais il peut aussi former une combinaison dans le seul but de vérifier une partie des conclusions des coups précédents et de faire en conséquence la proposition la plus propice à la déduction d'une nouvelle information.

Quand il est arrivé, au bout d'un certain nombre de coups, à placer les 4 pions qui correspondent exactement par la couleur et la position à ceux du code, la manche est terminée.

In [4]:
import random
import numpy as np
from itertools import product

TODO

In [5]:
def redlog(message):
    print(f"\033[91m{message}\033[0m")
def bluelog(message):
    print(f"\033[94m{message}\033[0m")



def evaluate_guess(guess, secret_code):
    exact = sum([1 for i in range(len(secret_code)) if guess[i] == secret_code[i]])
    return exact, sum([min(guess.count(j), secret_code.count(j)) for j in set(guess)]) - exact

TODO

In [6]:
class GuessNode:
    def __init__(self, parent, parameters, guess):
        self.parameters = parameters
        self.guess = guess

        self.children = []
        self.parent = parent

        self.possible_codes = parent.possible_codes.copy()
        self.history = parent.history.copy()
        self.moves = parent.moves + 1  # (node depth)

        self.total_moves = 0  # sum of moves of all children (for ucb1)
        self.visits = 0

    def expand(self):  # (add feedback children)
        if len(self.possible_codes) == 1:
            redlog("ERROR: extent of a terminal GuessNode")

        possible_feedbacks = {}

        for code in self.possible_codes:
            feedback = evaluate_guess(code, self.guess)

            if feedback not in possible_feedbacks:
                possible_feedbacks[feedback] = [code]
            else:
                possible_feedbacks[feedback].append(code)

        n = len(self.possible_codes)

        for feedback in possible_feedbacks:
            child = FeedbackNode(self.parameters, feedback=feedback, parent=self,
                                 frequency=len(possible_feedbacks[feedback]) / n)
            self.children.append(child)

    def update(self, n_moves):
        self.visits += 1
        self.total_moves += n_moves

    def score(self):  # (ucb1)
        if self.visits == 0:
            return float('inf')
        score = -self.total_moves / self.visits + np.sqrt(2 * np.log(self.parent.visits) / self.visits)
        return score

    def simulate(self):
        moves, possible_codes, guess = self.moves, self.possible_codes.copy(), self.guess
        secret_code = random.choice(self.possible_codes)
        while True:
            feedback = evaluate_guess(guess, secret_code)
            if feedback == (self.parameters["code_length"], 0):
                return moves
            new_possible_codes = []
            for code in possible_codes:
                if evaluate_guess(guess, code) == feedback:
                    new_possible_codes.append(code)
            possible_codes = new_possible_codes
            moves += 1
            guess = random.choice(possible_codes)

    def __str__(self):
        return f"GuessNode : {self.history} With guess: {self.guess}"

    def is_terminal(self):
        return len(self.possible_codes) == 1


class FeedbackNode:
    def __init__(self, parameters, feedback=None, parent=None, frequency=1.):
        self.parameters = parameters
        self.feedback = feedback
        self.frequency = frequency

        self.visits = 0

        if parent:
            self.moves = parent.moves
            self.guess = parent.guess
            self.history = parent.history.copy()
            self.history.append((parent.guess, feedback))
            self.possible_codes = [code for code in parent.possible_codes if
                                   evaluate_guess(self.guess, secret_code=code) == feedback]
        else:
            self.moves = 0
            self.guess = None
            self.history = []
            self.possible_codes = list(
                product(range(1, self.parameters["num_colors"] + 1), repeat=self.parameters["code_length"]))

        self.parent = parent
        self.children = []

        self.expand()

    def simulate(self):
        redlog("ERROR: FeedbackNode should not simulate")

    def expand(self):  # (add_guess_children)
        for code in self.possible_codes:
            child = GuessNode(self, self.parameters, code)
            self.children.append(child)

    def score(self):
        return self.frequency

    def update(self, moves):
        self.visits += 1

    def __str__(self):
        return f"FeedbackNode : {self.history}"

    def is_terminal(self):
        redlog("ERROR: GuessNode should have been caught first")
        return len(self.possible_codes) == 1

In [7]:
def select(root):
    guess_nodes = root.children
    chosen_guess_node = max(guess_nodes, key=lambda guess_node: guess_node.score())

    if chosen_guess_node.children:
        feedback_nodes = chosen_guess_node.children
        frequencies = [node.frequency for node in feedback_nodes]
        total_frequency = np.sum(frequencies)
        normalized_probabilities = [freq / total_frequency for freq in frequencies]

        chosen_feedback_node = np.random.choice(feedback_nodes, p=normalized_probabilities)

        return chosen_feedback_node

    return chosen_guess_node

    while node.children:
        node = max(node.children, key=lambda child: child.ucb1())

    return node

In [8]:
def select(node):
    guess_nodes = node.children
    if not guess_nodes:
        return node
    chosen_guess_node = max(guess_nodes, key=lambda guess_node: guess_node.score())

    if chosen_guess_node.children:
        feedback_nodes = chosen_guess_node.children

        probas = [node.score() for node in feedback_nodes]
        chosen_feedback_node = np.random.choice(feedback_nodes, p=probas)
        return select(chosen_feedback_node)

    return chosen_guess_node


def backpropagate(node, total_moves):
    while node is not None:
        node.update(total_moves)
        node = node.parent

In [9]:
def mcts_training(num_iterations):
    parameters = {
        "code_length": 2,
        "num_colors": 3
    }
    root = FeedbackNode(parameters)
    for i in range(num_iterations+1):
        if i % 100 == 0:
            bluelog(f"\nITERATION {i}/{num_iterations}")
        guess_node = select(root)
        if guess_node.is_terminal():
            total_moves = guess_node.moves

        else:
            guess_node.expand()
            total_moves = guess_node.simulate()
        backpropagate(guess_node, total_moves)
    return root

In [10]:
root = mcts_training(10000)


ITERATION 0/10000

ITERATION 100/10000

ITERATION 200/10000

ITERATION 300/10000

ITERATION 400/10000

ITERATION 500/10000

ITERATION 600/10000

ITERATION 700/10000

ITERATION 800/10000

ITERATION 900/10000

ITERATION 1000/10000

ITERATION 1100/10000

ITERATION 1200/10000

ITERATION 1300/10000

ITERATION 1400/10000

ITERATION 1500/10000

ITERATION 1600/10000

ITERATION 1700/10000

ITERATION 1800/10000

ITERATION 1900/10000

ITERATION 2000/10000

ITERATION 2100/10000

ITERATION 2200/10000

ITERATION 2300/10000

ITERATION 2400/10000

ITERATION 2500/10000

ITERATION 2600/10000

ITERATION 2700/10000

ITERATION 2800/10000

ITERATION 2900/10000

ITERATION 3000/10000

ITERATION 3100/10000

ITERATION 3200/10000

ITERATION 3300/10000

ITERATION 3400/10000

ITERATION 3500/10000

ITERATION 3600/10000

ITERATION 3700/10000

ITERATION 3800/10000

ITERATION 3900/10000

ITERATION 4000/10000

ITERATION 4100/10000

ITERATION 4200/10000

ITERATION 4300/10000

ITERATION 4400/10000

ITERATION 4500/10000


In [11]:
def show_f(node_f, depth=1):
    print(f"{'  ' * depth}{node_f} ({node_f.visits} visits, frequency:{node_f.frequency})")
    for child in node_f.children:
        show_g(child, depth + 1)

def show_g(node_g, depth=1):
    print(f"{'  ' * depth}{node_g} ({node_g.visits} visits, {node_g.total_moves} total_moves)")
    for child in node_g.children:
        show_f(child, depth + 1)

show_f(root)

  FeedbackNode : [] (10001 visits, frequency:1.0)
    GuessNode : [] With guess: (1, 1) (83 visits, 253 total_moves)
      FeedbackNode : [((1, 1), (2, 0))] (11 visits, frequency:0.1111111111111111)
        GuessNode : [((1, 1), (2, 0))] With guess: (1, 1) (11 visits, 22 total_moves)
      FeedbackNode : [((1, 1), (1, 0))] (31 visits, frequency:0.4444444444444444)
        GuessNode : [((1, 1), (1, 0))] With guess: (1, 2) (9 visits, 26 total_moves)
          FeedbackNode : [((1, 1), (1, 0)), ((1, 2), (2, 0))] (0 visits, frequency:0.25)
            GuessNode : [((1, 1), (1, 0)), ((1, 2), (2, 0))] With guess: (1, 2) (0 visits, 0 total_moves)
          FeedbackNode : [((1, 1), (1, 0)), ((1, 2), (1, 0))] (3 visits, frequency:0.25)
            GuessNode : [((1, 1), (1, 0)), ((1, 2), (1, 0))] With guess: (1, 3) (3 visits, 9 total_moves)
          FeedbackNode : [((1, 1), (1, 0)), ((1, 2), (0, 2))] (1 visits, frequency:0.25)
            GuessNode : [((1, 1), (1, 0)), ((1, 2), (0, 2))] With gue